## Uploading Data to Postgres (Data1.ipynb)

The purpose of this workbook is take our original data sources and save them within our Postgres database.  By having the data centralized in our Postgres database, we are able to facilitate collaboration.  In order to achieve this, we have the following high-level steps:
#### 1. [Confirm connectivity to our team database](#data_connect)
#### 2. [Importing raw data in to the the Postgres database and schema](#data_import)
#### 3. [Verify successful insert](#data_verify)

We also have some initial exploration in to working with [wide data](#data_wide) (1600+ columns) and how we can break that in to smaller tables within Postgres

### <a name="data_connect"></a>Confirm connectivity to our team database
Since multiple team members have had previous experience with Python, we decided to proceed with using that for our project.

Through the use psycopg2, sqlalchemy, and getpass we can securely test our connection to the database.  The below code is used to verify that a specific username is able to access a specific database.
1. Username: dtfp3
2. Database: pgsql.dsa.lan/casestdysu23t03

In [1]:
# Import all necessary libraries for this notebook
import pandas as pd
import binascii
import psycopg2
import sqlalchemy
import getpass

In [8]:
# Since we were able to successfully access, I'm commenting it out
user = "dtfp3" # Make sure your username is correct 
host = "pgsql.dsa.lan"
database = "casestdysu23t03"
password = getpass.getpass()
connectionstring = "postgresql://" + user + ":" + password + "@" + host + "/" + database
engine = sqlalchemy.create_engine(connectionstring)
connection = None
try:
    connection = engine.connect()
    print("Successfully connected to {}/{} as {}".format(host, database,user))
except Exception as err:
    print("An error has occurred trying to connect: {}".format(err))
    
del password

········
Successfully connected to pgsql.dsa.lan/casestdysu23t03 as dtfp3


### <a name="data_import"></a>Importing raw data in to the the Postgres database and schema
Our primary data source is from [The Osteoporotic Fractures in Men (MrOS) Study](https://mrosonline.ucsf.edu/).  After signing up for a free account, we were then able to download the raw data files.  We have stored these files in our DSA file directory:  **/dsa/groups/casestudy2023su/team03**


Since the raw data files are stored as SAS7BDAT, many different methods require the additional step of converting them to csv for readability and import.  <br>
`df = pd.read_sas(sas_filename)
binary2StringLiteral(df)
df.to_csv(csv_filename, index=False)`
    
After the files have been converted, we then need to start our import process.  An important first step is understanding whether the tables already exist or not.  Since we're starting from the raw data, we're:
1. **SELECT** to determine whether the tables exist or not
    1. If it does exist, the table will be **DROP**ed
1. **CREATE** the table - we're creating these tables with a few considerations
    1. The table will be created a concatenated string equaling filename+<_raw>
    2. We'll use the datatypes from the CSV so everyting isn't casted as a string
    3. Postgres has a limit of 1600 columns so if it's greater than that, we store it as a single column that can be parsed later
1. **GRANT** all **PRIVILEGES** to users within the **PUBLIC** group so everyone can manipulate the tables 
1. **INSERT** the data in to the respective columns
       
As a note, it's possible for a user to have a table-lock which would prevent the tables from being dropped. This can be resolved by checking who has the active transaction and then terminating or cancelling it from either the psql terminal or it could be done through another notebook connection:

**Identify the table lock** <br>
`select datname, pid, usename, application_name, state, query_start  from pg_stat_activity where datname = 'casestdysu23t03';`

**Terminate the PID** <br>
`SELECT pg_terminate_backend(23174);`

In [3]:
# Alright, now for the fun stuff... 
user = "dtfp3"
host = "pgsql.dsa.lan"
database = "casestdysu23t03"
password = getpass.getpass()
schema = "public" # Started with a different schema and then upated to public

# List of SAS files that the team will be uploading
sas_files = [
    #"ASGAUG14.SAS7BDAT",
    "B1AUG16.SAS7BDAT",
    "FAFEB23.SAS7BDAT",
    #"GNAUG15.SAS7BDAT",
    "V1FEB23.SAS7BDAT",
    "B2AUG16.SAS7BDAT" # Adding B2AGU16 data on 7/4/23
]

dtype2SQL = {'object' : 'TEXT', 'float64' : 'REAL', 'int64' : "INTEGER","datetime64[ns]":'TEXT'}
##dtype2format = {'object' : r"%s", 'float64' : r'%', 'int64' : r"%i","datetime64[ns]":r'%s'}

# Function to convert binary data to string literals since it's possible we'll encounter it within the files
def binary2StringLiteral(df):
    for column in df.columns:
        if df[column].dtype == "object":
            df[column] = df[column].str.decode('utf-8')

# Connection setup
connection = None

try:
    connection = psycopg2.connect(user=user, host=host, database=database, password=password)
    cursor = connection.cursor()

    for sas_file in sas_files:
        # Convert SAS7BDAT to CSV
        sas_filename = f"/dsa/groups/casestudy2023su/team03/{sas_file}"
        csv_filename = f"/dsa/groups/casestudy2023su/team03/{sas_file}.csv"

        df = pd.read_sas(sas_filename)
        binary2StringLiteral(df)
        df.to_csv(csv_filename, index=False)

        # We're dealing with the raw data that could be very messy - starting with _raw for clarity
        table_name = sas_file.split(".")[0].lower() + "_raw"

        # Check if table exists
        check_table_query = f"SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_schema = '{schema}' AND table_name = '{table_name}')"
        cursor.execute(check_table_query)
        table_exists = cursor.fetchone()[0]

        if table_exists:
            print("Table already exists - blowing it away: {}".format(table_name))
            # Drop the table if it exists
            drop_table_query = f"DROP TABLE {schema}.{table_name}"
            cursor.execute(drop_table_query)
            connection.commit()

        # Create the table
        print("Creating Table: {}".format(table_name))
        
        if len(df.columns) >= 1600:
            # If the number of columns is equal to or more than 1600, create a table with a single "data_column"
            create_table_query = f"CREATE TABLE {schema}.{table_name} (data_column text)"
            cursor.execute(create_table_query)
        else:
            # If the number of columns is less than 1600, create a table with all the columns from the CSV
            columns = ', '.join([f'"{col}" {dtype2SQL[str(df[col].dtype)]}' for col in df.columns])
            create_table_query = f"CREATE TABLE {schema}.{table_name} ({columns})"
            cursor.execute(create_table_query)
        connection.commit()

        # Grant necessary privileges to all users
        grant_query = f"GRANT ALL PRIVILEGES ON TABLE {schema}.{table_name} TO PUBLIC"
        cursor.execute(grant_query)
        connection.commit()
        
        print(f"Table {schema}.{table_name} created successfully.")

        # Turns out we don't have permissions on public to use copy
        # copy_query = f"COPY {schema}.{table_name}(data_column) FROM '{csv_filename}' DELIMITER ',' CSV"

        # Alernative is to read in the CSV file and insert the data
        # A bit slower... but it works! 
        with open(csv_filename, 'r') as file:
            if len(df.columns) >= 1600:
                # We don't skip the header because we need that data if we're building another table. 
                # If the number of columns is equal to or more than 1600, read in each row but don't parse the columns
                for line in file:
                    data_column = line.strip()
                    insert_query = f"INSERT INTO {schema}.{table_name} (data_column) VALUES (%s)"
                    cursor.execute(insert_query, (data_column,))
            else:
                # Skip header row
                next(file)
                
                # If the number of columns is less than 1600, insert the data row by row
                for line in file:
                    values = line.strip().split(',')
                    values = [None if x == "" else x for x in values]
                    insert_query = f"INSERT INTO {schema}.{table_name} VALUES ({','.join(['%s']*len(values))})"
                    cursor.execute(insert_query, values)

        connection.commit()

        print(f"File {sas_file} inserted successfully into table {schema}.{table_name}.")

    print("All files inserted successfully.")

except Exception as err:
    print("An error has occurred: {}".format(err))

finally:
    if connection:
        cursor.close()
        connection.close()

del password


········
Table already exists - blowing it away: b1aug16_raw
Creating Table: b1aug16_raw
Table public.b1aug16_raw created successfully.
File B1AUG16.SAS7BDAT inserted successfully into table public.b1aug16_raw.
Table already exists - blowing it away: fafeb23_raw
Creating Table: fafeb23_raw
Table public.fafeb23_raw created successfully.
File FAFEB23.SAS7BDAT inserted successfully into table public.fafeb23_raw.
Table already exists - blowing it away: v1feb23_raw
Creating Table: v1feb23_raw
Table public.v1feb23_raw created successfully.
File V1FEB23.SAS7BDAT inserted successfully into table public.v1feb23_raw.
Creating Table: b2aug16_raw
Table public.b2aug16_raw created successfully.
File B2AUG16.SAS7BDAT inserted successfully into table public.b2aug16_raw.
All files inserted successfully.


### <a name="data_verify"></a>Verify successful insert

Now we have the opportunity to see whether the inserts were succesful. 
1. Connect to the database again
2. Select data from the B1AUG16_raw to test

In [4]:
# Time to hop back in and let's see how things are looking!
password = getpass.getpass('Type Password and hit enter :: ')

connection = psycopg2.connect(database = 'casestdysu23t03', 
                              user = 'dtfp3', 
                              host = 'pgsql.dsa.lan',
                              password = password)
del password


# Query column information
with connection, connection.cursor() as cursor:
    cursor.execute('''SELECT column_name, data_type
                    FROM information_schema.columns
                    WHERE table_schema = 'public' AND 
                    table_name = 'b1aug16_raw';''')
    results = cursor.fetchall()

    print("data from B1AUG16 table")
    i = 0
    for row in results:
        i += 1
        print(str(i) + str(row))
        


Type Password and hit enter :: ········
data from B1AUG16 table
1('B1HPQDR', 'text')
2('B1TRA', 'real')
3('B1TRC', 'real')
4('B1TRD', 'real')
5('B1ITA', 'real')
6('B1ITC', 'real')
7('B1ITD', 'real')
8('B1FNA', 'real')
9('B1FNC', 'real')
10('B1FND', 'real')
11('B1WDA', 'real')
12('B1WDC', 'real')
13('B1WDD', 'real')
14('B1THA', 'real')
15('B1THC', 'real')
16('B1THD', 'real')
17('B1HPADT', 'text')
18('B1HPDATE', 'text')
19('B1HPTYP', 'real')
20('B1HPMODE', 'real')
21('B1HPCOD', 'text')
22('B1HPPRO', 'real')
23('B1HPSIDE', 'text')
24('B1HPID', 'text')
25('SITE', 'text')
26('ID', 'text')
27('B1SPQDR', 'text')
28('B1L1A', 'real')
29('B1L1C', 'real')
30('B1L1D', 'real')
31('B1L2A', 'real')
32('B1L2C', 'real')
33('B1L2D', 'real')
34('B1L3A', 'real')
35('B1L3C', 'real')
36('B1L3D', 'real')
37('B1L4A', 'real')
38('B1L4C', 'real')
39('B1L4D', 'real')
40('B1TLA', 'real')
41('B1TLC', 'real')
42('B1TLD', 'real')
43('B1SPADT', 'text')
44('B1SPDATE', 'text')
45('B1SPTYP', 'real')
46('B1SPMODE', 'real

### <a name="data_wide"></a>Working with wide data
Since we are working with very wide data, we had to account for that when doing our insert in to Postgres.  The **fafeb23_raw** table we created with greater than 1600 columns was stored as a single column.  

This is an exploratory option of selecting and then creating a new table. **fafeb23_fx** with only a subset of the columns.  Depending on our EDA and analysis, we may choose to refactor this code to select different columns and create additional tables.  The runs through similar steps to above:
1. Droping the fafeb23_fx table if it already exists
2. Take the first row of the fafeb23_raw table to capture all the column names
3. Retain only the column names that we'll be using for building our new table
    1. In the case, we're only retaining the id, site, and "fx" columns
4. Create the new table
5. Grant the privileges on the new table
6. Insert the row of data with Pandas

In [10]:
# Define the new table information
schema = "public"
new_table_name = 'fafeb23_fx'
original_table_name = 'fafeb23_raw'

# Creating a new table, fafeb23_fx, from the public.fafeb23_raw table
with connection, connection.cursor() as cursor:

    drop_table_query = f"DROP TABLE {schema}.{new_table_name}"
    cursor.execute(drop_table_query)
    connection.commit()

    
    # Retrieve the first row from the table
    row_query = f"SELECT * FROM {schema}.{original_table_name} LIMIT 1"
    cursor.execute(row_query)
    first_row = cursor.fetchone()[0]
    #print(first_row)
    
    # Specify the desired columns to keep
    columns_to_keep = ['id', 'site']
    columns_containing_fx = [col for col in first_row.split(',') if 'fx' in col.lower()]
    #print(columns_containing_fx)
    
    # Convert column names to lowercase
    columns_to_keep = [col.lower() for col in columns_to_keep]
    columns_containing_fx = [col.lower() for col in columns_containing_fx]

    # Generate the column names for the new table
    column_names = columns_to_keep + columns_containing_fx
    print(column_names)

    column_definitions = ', '.join([f'"{col}" text' for col in column_names])

    # Create the new table
    create_table_query = f"CREATE TABLE {new_table_name} ({column_definitions})"
    cursor.execute(create_table_query)
    connection.commit()

    # Grant necessary privileges to all users
    grant_query = f"GRANT ALL PRIVILEGES ON TABLE public.fafeb23_fx TO PUBLIC"
    cursor.execute(grant_query)
    connection.commit()
    
# Read the CSV file associated with the desired columns
# Need to make them uppercase though...
column_names = [col.upper() for col in column_names]

csv_filename = f'/dsa/groups/casestudy2023su/team03/FAFEB23.SAS7BDAT.csv'
df = pd.read_csv(csv_filename, usecols=column_names)

# Insert data into the new table
with connection, connection.cursor() as cursor:
    # Iterate over the rows of the DataFrame
    for _, row in df.iterrows():
        values = [str(row[col]) for col in column_names]
        insert_query = f"INSERT INTO {new_table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s'] * len(column_names))})"
        cursor.execute(insert_query, values)
    connection.commit()

AttributeError: 'Connection' object has no attribute 'cursor'